<a href="https://colab.research.google.com/github/virf96/Proyectos/blob/master/Auto_Arima.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Packages

In [1]:
!pip install arch

     |████████████████████████████████| 798kB 3.4MB/s 


In [2]:
!pip install pmdarima

     |████████████████████████████████| 1.5MB 3.3MB/s 
     |████████████████████████████████| 8.7MB 17.3MB/s 
     |████████████████████████████████| 2.1MB 41.7MB/s 
  Found existing installation: statsmodels 0.10.2
    Uninstalling statsmodels-0.10.2:
      Successfully uninstalled statsmodels-0.10.2
  Found existing installation: Cython 0.29.21
    Uninstalling Cython-0.29.21:
      Successfully uninstalled Cython-0.29.21


In [3]:
!pip install yfinance

  Created wheel for yfinance: filename=yfinance-0.1.54-py2.py3-none-any.whl size=22409 sha256=5a9c6906e4f1dba11b903436bfe8cf2ea576a920c8e719cf926a77f8cca094b9
  Stored in directory: /root/.cache/pip/wheels/f9/e3/5b/ec24dd2984b12d61e0abf26289746c2436a0e7844f26f2515c
Successfully built yfinance


In [14]:
import numpy as np
import pandas as pd
import scipy
import statsmodels.api as sm
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
from statsmodels.tsa.arima_model import ARIMA
from arch import arch_model
import yfinance
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")
sns.set()

### Loading the data

In [21]:
raw_data = yfinance.download (tickers = "^GSPC ^FTSE ^N225 ^GDAXI", start = "1994-01-07", end = "2020-03-20", 
                              interval = "1d", group_by = 'ticker', auto_adjust = True, treads = True)

[*********************100%***********************]  4 of 4 completed


In [22]:
# Hacemos una copa de la información
df_comp = raw_data.copy()

In [23]:
df_comp.head()

^N225                ...       ^GSPC             
                    Open          High  ...       Close       Volume
Date                                    ...                         
1994-01-07  17842.980469  18131.410156  ...  469.899994  324920000.0
1994-01-10  18186.519531  18567.060547  ...  475.269989  319490000.0
1994-01-11  18481.849609  18671.669922  ...  474.130005  305490000.0
1994-01-12  18447.339844  18807.080078  ...  474.170013  310690000.0
1994-01-13  18770.380859  18823.380859  ...  472.470001  277970000.0

[5 rows x 20 columns]

In [24]:
#Nos quedamos unicamente con el precio de cierre
df_comp['spx'] = df_comp['^GSPC'].Close[:]
df_comp['dax'] = df_comp['^GDAXI'].Close[:]
df_comp['ftse'] = df_comp['^FTSE'].Close[:]
df_comp['nikkei'] = df_comp['^N225'].Close[:]

In [19]:
df_comp.head()

^FTSE               ...         ftse        nikkei
                   Open         High  ...                           
Date                                  ...                           
1994-01-07  3401.399902  3446.800049  ...  3446.000000  18124.009766
1994-01-10  3465.699951  3468.100098  ...  3440.600098  18443.439453
1994-01-11  3442.500000  3442.500000  ...  3413.800049  18485.250000
1994-01-12  3394.800049  3402.399902  ...  3372.000000  18793.880859
1994-01-13  3380.699951  3383.300049  ...  3360.000000  18577.259766

[5 rows x 24 columns]

In [25]:
df_comp.tail()

^N225                ...         ftse        nikkei
                    Open          High  ...                           
Date                                    ...                           
2020-03-13  18183.470703  18184.460938  ...  5366.100098  17431.050781
2020-03-16  17586.080078  17785.759766  ...  5151.100098  17002.039062
2020-03-17  16726.949219  17557.039062  ...  5294.899902  17011.529297
2020-03-18  17154.080078  17396.839844  ...  5080.600098  16726.550781
2020-03-19  16995.769531  17160.970703  ...  5151.600098  16552.830078

[5 rows x 24 columns]

In [26]:
#Borramos las variables que no nos sirven
del df_comp['^N225']
del df_comp['^GSPC']
del df_comp['^GDAXI']
del df_comp['^FTSE']
#Establecemos la periodicidad como "business days"
df_comp=df_comp.asfreq('b')
#Y hacemos el relleno de NA's con el método front fill
df_comp=df_comp.fillna(method='ffill')

In [27]:
df_comp.head()

spx          dax         ftse        nikkei
                                                              
Date                                                          
1994-01-07  469.899994  2224.949951  3446.000000  18124.009766
1994-01-10  475.269989  2225.000000  3440.600098  18443.439453
1994-01-11  474.130005  2228.100098  3413.800049  18485.250000
1994-01-12  474.170013  2182.060059  3372.000000  18793.880859
1994-01-13  472.470001  2142.370117  3360.000000  18577.259766

### Creating Returns

In [28]:
#Creamos el retorno de las variables para trabajar con estas
df_comp['ret_spx'] = df_comp.spx.pct_change(1)*100
df_comp['ret_ftse'] = df_comp.ftse.pct_change(1)*100
df_comp['ret_dax'] = df_comp.dax.pct_change(1)*100
df_comp['ret_nikkei'] = df_comp.nikkei.pct_change(1)*100

### Splitting the Data

In [29]:
size = int(len(df_comp)*0.8)
df, df_test = df_comp.iloc[:size], df_comp.iloc[size:]

### Fitting a Model

In [30]:
from pmdarima.arima import auto_arima

In [31]:
#Tomamos los retornos de ftse desde la fila 1 dado que al tener los retornos, la fila 0 es NA
#Generamos un modelo según lo recomendado por "auto arima"
model_auto = auto_arima(df.ret_ftse[1:])

In [32]:
#Imprimimos el modelo ajustado por auto arima
#Analizando vemos que tenemos un modelo que tiene 3 componentes autoregresivos, 0 integraciones y 5 componentes de medias móviles
#No tenemos nada en el orden estacional
#Concluimos que la función ajusto un modelo ARMA(3,5)
model_auto

ARIMA(maxiter=50, method='lbfgs', order=(3, 0, 5), out_of_sample_size=0,
      scoring='mse', scoring_args={}, seasonal_order=(0, 0, 0, 0),
      start_params=None, suppress_warnings=False, trend=None,
      with_intercept=True)

In [33]:
#Aun que el modelo diga SARIMAX, es evidente que es un modelo ARMA(3,5)
model_auto.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                               SARIMAX Results                                
==============================================================================
Dep. Variable:                      y   No. Observations:                 5467
Model:               SARIMAX(3, 0, 5)   Log Likelihood               -8452.784
Date:                Wed, 12 Aug 2020   AIC                          16925.568
Time:                        02:55:07   BIC                          16991.633
Sample:                             0   HQIC                         16948.618
                               - 5467                                         
Covariance Type:                  opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
intercept      0.0439      0.033      1.322      0.186      -0.021       0.109
ar.L1         -0.1553      0.075     -2.069      0.039      -0.302      -0.008
ar.L2         -0.8711      0.019    -45.802      0.000      -0.908      -0.834
ar.L3         -0.3071      0.069     -4.453      0.000      -0.442      -0.172
ma.L1          0.1314      0.075      1.747      0.081      -0.016       0.279
ma.L2          0.8176      0.022     37.485      0.000       0.775       0.860
ma.L3          0.1992      0.068      2.937      0.003       0.066       0.332
ma.L4         -0.0105      0.009     -1.210      0.226      -0.028       0.007
ma.L5         -0.1076      0.008    -13.354      0.000      -0.123      -0.092
sigma2         1.2872      0.013     96.144      0.000       1.261       1.313
===================================================================================
Ljung-Box (Q):                       75.56   Jarque-Bera (JB):              7320.10
Prob(Q):                              0.00   Prob(JB):                         0.00
Heteroskedasticity (H):               1.81   Skew:                            -0.17
Prob(H) (two-sided):                  0.00   Kurtosis:                         8.66
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""

### Important Arguments

In [34]:
#Pongamos restricciones al ajuste del modelo para poder obtener un modelo más confiable y no tan general
model_auto2 = auto_arima(df_comp.ret_ftse[1:], exogenous = df_comp[['ret_spx', 'ret_dax', 'ret_nikkei']][1:], m = 5,
                       max_order = None, max_p = 7, max_q = 7, max_d = 2, max_P = 4, max_Q = 4, max_D = 2,
                       maxiter = 50, alpha = 0.05, n_jobs = -1, trend = 'ct', information_criterion = 'oob',
                       out_of_sample_size = int(len(df_comp)*0.2))


# exogenous -> factores externos (e.j otras series de tiempo)

#NOTA: debemos ser cuidadosos en los factores externos, también deben ser estacionales al igual que la variable endogena

# m -> tamaño del cliclo estacional
# max_order -> máxima suma de los ordenes de AR y MA en la serie de tiempo (p + q) restringe las combinaciones
# max_p -> máximos componentes en AR
# max_q -> máximos componentes en MA
# max_d -> máximos componentes de integración 
# maxiter -> máximo número de iteraciones antes de que el modelo converja (se vuelve más difícil a medida que aumenta el orden)
# alpha -> nivel de significancia, por default se escoje el 5%, ue deberíamos usar la mayor parte del tiempo
# n_jobs -> cuantos modelos debemos ajustar al mismo tiempo  (-1 indica"tantos como sea posible")
# trend -> "ct" usualmente, en caso de que tendencial cuadratica debería ser "ctt"
# information_criterion -> 'aic', 'aicc', 'bic', 'hqic', 'oob' 
#        (Akaike Information Criterion, Corrected Akaike Information Criterion,
#        Bayesian Information Criterion, Hannan-Quinn Information Criterion, or
#        "out of bag"--for validation scoring--respectively)
# out_of_smaple_size -> valida la selección del modelo (pase el conjunto de datos completo y establezca el 20% para que sea out_of_sample_size)

In [35]:
# observamos que
model_auto2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                     SARIMAX Results                                      
==========================================================================================
Dep. Variable:                                  y   No. Observations:                 6834
Model:             SARIMAX(0, 0, 1)x(4, 0, [], 5)   Log Likelihood               -6809.273
Date:                            Wed, 12 Aug 2020   AIC                          13640.547
Time:                                    03:27:14   BIC                          13715.673
Sample:                                         0   HQIC                         13666.465
                                           - 6834                                         
Covariance Type:                              opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
intercept     -0.0008      0.015     -0.052      0.959      -0.031       0.029
drift      -2.401e-06   4.33e-06     -0.555      0.579   -1.09e-05    6.08e-06
x1             0.0872      0.006     15.219      0.000       0.076       0.098
x2             0.5624      0.005    110.721      0.000       0.552       0.572
x3             0.0720      0.004     16.452      0.000       0.063       0.081
ma.L1         -0.1165      0.008    -15.498      0.000      -0.131      -0.102
ar.S.L5       -0.0332      0.009     -3.776      0.000      -0.050      -0.016
ar.S.L10      -0.0503      0.010     -5.225      0.000      -0.069      -0.031
ar.S.L15      -0.0349      0.009     -3.844      0.000      -0.053      -0.017
ar.S.L20      -0.0233      0.009     -2.521      0.012      -0.041      -0.005
sigma2         0.4641      0.005     99.300      0.000       0.455       0.473
===================================================================================
Ljung-Box (Q):                       76.91   Jarque-Bera (JB):             15206.90
Prob(Q):                              0.00   Prob(JB):                         0.00
Heteroskedasticity (H):               0.48   Skew:                             0.22
Prob(H) (two-sided):                  0.00   Kurtosis:                        10.29
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""